In [1]:
from wand.image import Image
from PIL import Image as pil
import numpy as np
import pandas as pd
import io
import matplotlib.pyplot as plt

In [2]:
#答案用紙のサイズが違うとき、補完する関数
def filledup(array):
    if array.shape[0] < 845:
        supplement = np.ones((845-array.shape[0], array.shape[1]))
        array = np.vstack((array, supplement))
        
    if array.shape[1] < 595:
        supplement = np.ones((array.shape[0], 595-array.shape[1]))
        array = np.hstack((array, supplement))
        
    return array

In [3]:
#解答用紙を画素ベクトルに変換する関数
def convert_to_array(filename, pix):

    with Image(filename=filename) as f:
        with Image(image=f.sequence[0]) as page1:
            page1 = page1.convert("jpeg")
            page1.save(filename=r"C:\Users\Aki\Documents\saiten\928\test.jpg")
            img = page1.make_blob("jpeg")
            img_array = np.array(pil.open(io.BytesIO(img)))
            dim = len(img_array.shape)
            
            #次元の異なる画像に対応
            if dim == 3:
                img_array = np.sum(img_array, axis=2) / 765
            elif dim == 2:
                img_array = img_array / 255
                
            #解像度の小さい解答用紙への対応
            img_array = filledup(img_array)
            print(img_array.shape)
            
            #図への描画
            #indexes = np.where(img_array<0.5)
            #plt.scatter(indexes[0], indexes[1], s=1)
            #plt.title(filename)
            #plt.show()
            
            #0のサンプルが多すぎるので削減
            img_array = img_array[:pix]
            
    return img_array

In [4]:
#学習させる答案用紙の指定
train_file = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501430715652.pdf", 200)
y1 = np.zeros(train_file.shape[0], dtype=np.int8)

for i in range(119, 126):
    y1[i] = 1

(845, 595)


In [5]:
train_file1 = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501631716385.pdf", 200)
y2 = np.zeros(train_file1.shape[0], dtype=np.int8)

for i in range(128, 135):
    y2[i] = 1

(845, 595)


In [6]:
train_file2 = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501530355427.pdf", 200)
y3 = np.zeros(train_file2.shape[0], dtype=np.int8)

for i in range(125, 132):
    y3[i] = 1

(845, 595)


In [7]:
train_file3 = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501530406463.pdf",200)
y4 = np.zeros(train_file3.shape[0], dtype=np.int8)

for i in range(119,126):
    y4[i] = 1

(845, 595)


In [8]:
train_file4 = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501628151097.pdf", 200)
y5 = np.zeros(train_file4.shape[0])

for i in range(118, 126):
    y5[i] = 1

(845, 595)


In [9]:
train_file5 = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501628491262.pdf", 200)
y6 = np.zeros(train_file5.shape[0])

for i in range(119, 126):
    y6[i] = 1

(845, 595)


In [10]:
train_file6 = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501629885060.pdf", 200)
y7 = np.zeros(train_file6.shape[0])

for i in range(101, 108):
    y7[i] = 1

(845, 595)


In [11]:
train_file7 = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501629257212.pdf", 200)
y8 = np.zeros(train_file7.shape[0])

(1224, 842)


In [12]:
#データの連結
X = np.vstack((train_file, train_file1, train_file2, train_file3, train_file4, train_file5, train_file6))
y = np.hstack((y1, y2, y3, y4, y5, y6, y7))

print(X.shape, y.shape)

(1400, 595) (1400,)


In [13]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=10)

In [14]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(model, X, y, cv=kf, scoring="recall_macro")
print("再現率は{}です。".format(scores.mean()))

再現率は0.8196296296296296です。


In [15]:
#trainデータのscore
print(model.score(X, y))

NotFittedError: Estimator not fitted, call `fit` before exploiting the model.

In [14]:
#testデータのscore
X_test = convert_to_array(r"C:\Users\Aki\Documents\saiten\928\022617501528337191.pdf",845)
y_test = np.zeros(X_test.shape[0])

(845, 595)


In [15]:
for i in range(105, 113):
    y_test[i] = 1

In [16]:
test_score = model.score(X_test, y_test)

In [17]:
print(test_score)

0.996449704142


In [18]:
y_predicted = model.predict(X_test)
indexes = np.where(y_predicted==1)
print(indexes)

(array([107, 108, 109, 110, 111, 112, 231], dtype=int32),)


In [19]:
#混同行列と再現率の出力
import sklearn.metrics as metrics
cm = metrics.confusion_matrix(y_test, y_predicted)
recall = metrics.recall_score(y_test, y_predicted)
print(cm)
print("recall = {}".format(recall))

[[836   1]
 [  2   6]]
recall = 0.75
